In [2]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# System related and data input controls
import os

# Auto reload of library
%reload_ext autoreload
%autoreload 2

# Python path
import sys
base_folder = 'DataScience'
location_base = os.path.join(os.getcwd().split(base_folder)[0], base_folder)
location_module = [os.path.join(location_base, 'Module')] 
for each in location_module:
    if each not in sys.path:
        sys.path.append(each)

from import_KK import *
DeviceStrategy_GPU()
from data_KK import *
from description_KK import *
from preprocessing_KK import *
from preprocessing_text_KK import * ##
from algorithm_textmining_KK import * ##
from evaluation_KK import *
from visualization_KK import *

FONT_NAME = 'malgun'
plt.rc('font', family=FONT_NAME) 
plt.rcParams['font.family'] = FONT_NAME
mpl.rc('font', family=FONT_NAME)
sns.set(font=FONT_NAME) 
sys_font = font_manager.findSystemFonts()
FONT_PATHS = [path for path in sys_font if 'malgun' in path]
if len(FONT_PATHS) != 0:
    rc('font', family=font_manager.FontProperties(fname=FONT_PATHS[0]).get_name())

Cuda is Ready?  True
Cuda Version:  12.5.1
Cudnn Version:  9 

Tensorflow Version:  2.18.0
Keras Version:  3.7.0
Torch Version:  2.5.0+cu124
Torch Cuda Version: 12.4
Torch Cudnn Version:90600 

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 4070 Laptop GPU


2025-02-25 21:46:47,840	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 
[nltk_data] Downloading package stopwords to /home/kk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Sentiment Analysis (3 Labels)

In [ ]:
# 데이터 파라미터
SOURCE_NAME = 'df_prep_(1,2).csv'
COLNAME_LABEL = 'Rating_Multi'
COLNAME_TARGET = 'Comment_Prep'
# 전처리모델 파라미터
# MODEL_NAME = 'tabularisai/multilingual-sentiment-analysis'
# MODEL_NAME = 'daekeun-ml/koelectra-small-v3-nsmc'
# MODEL_NAME = 'klue/bert-base'
# MODEL_NAME = 'hun3359/klue-bert-base-sentiment'
MODEL_NAME = 'kykim/bert-kor-base'
# MODEL_NAME = 'beomi/kcbert-base'
BINS = [0,3,8,10]
LABELS = list(range(len(BINS)))[:-1]
VAL_SIZE = 0.2
RANDOM_SEED = 123
WORD_MAXLEN = 512
SAMPLE_SIZE = None
EPOCHS = 10
BATCH = 256
LOSS = 'accuracy'
EVAL_STRATEGY = 'epoch'
OUTPUT_DIR = 'outputs'
# 예측 파라미터
TASK = 'text-classification'
SAVE_PATH_MODEL = os.path.join('.', 'Model', MODEL_NAME.split('/')[1]+'_'+str(len(LABELS))+'Label_'+SOURCE_NAME.split('.')[0])

In [ ]:
# Data Loading
df = pd.read_csv(os.path.join('.', 'Data', SOURCE_NAME), encoding='utf-8-sig')
df[COLNAME_LABEL] = feature_num_CategBin(df['Rating'], bins=BINS, labels=LABELS)
ds = preprocessing_df2Datasets(MODEL_NAME, df[[COLNAME_LABEL, COLNAME_TARGET]], 
                               label_list=LABELS, val_size=VAL_SIZE, 
                               random_seed=RANDOM_SEED, max_length=WORD_MAXLEN)

In [ ]:
# Modeling
trainer, history = modeling_SequenceClassification(MODEL_NAME, 
                                                   ds, sample_size=SAMPLE_SIZE,
                                                   train_epochs=EPOCHS,
                                                   loss=LOSS,
                                                   eval_strategy=EVAL_STRATEGY,
                                                   output_dir=OUTPUT_DIR,
                                                   save_folder=SAVE_PATH_MODEL)    

# Prediction
test_pred = prediction_SequenceClassification(task=TASK, model_path=SAVE_PATH_MODEL, 
                                              ds_test_text=ds['test']['text'])
## Replacing
for idx, pred in zip(df[df[COLNAME_LABEL] == -1].index, test_pred):
    df.loc[idx, COLNAME_LABEL] = pred[0]
    df.loc[idx, COLNAME_LABEL+'_Prob'] = pred[1]
df.to_csv(os.path.join('.', 'Data', SOURCE_NAME.split('.')[0]+'_pred('+MODEL_NAME.split('/')[1]+'_'+str(len(LABELS))+'Label).csv'), 
          encoding='utf-8-sig')

,count
Rating,
0,5193
1,20167
2,5315
-1,30128


Map:   0%|          | 0/24540 [00:00<?, ? examples/s]

Map:   0%|          | 0/6135 [00:00<?, ? examples/s]

Map:   0%|          | 0/30128 [00:00<?, ? examples/s]

# Sentiment Analysis (2 Labels)

In [39]:
# 데이터 파라미터
SOURCE_NAME = 'df_prep_(1,2).csv'
COLNAME_LABEL = 'Rating_Multi'
COLNAME_TARGET = 'Comment_Prep'
# 전처리모델 파라미터
# MODEL_NAME = 'tabularisai/multilingual-sentiment-analysis'
# MODEL_NAME = 'daekeun-ml/koelectra-small-v3-nsmc'
# MODEL_NAME = 'klue/bert-base'
# MODEL_NAME = 'hun3359/klue-bert-base-sentiment'
MODEL_NAME = 'kykim/bert-kor-base'
# MODEL_NAME = 'beomi/kcbert-base'
BINS = [0,5,10]
LABELS = list(range(len(BINS)))[:-1]
VAL_SIZE = 0.2
RANDOM_SEED = 123
WORD_MAXLEN = 512
SAMPLE_SIZE = None
EPOCHS = 10
BATCH = 256
LOSS = 'accuracy'
EVAL_STRATEGY = 'epoch'
OUTPUT_DIR = 'outputs'
# 예측 파라미터
TASK = 'text-classification'
SAVE_PATH_MODEL = os.path.join('.', 'Model', MODEL_NAME.split('/')[1]+'_'+str(len(LABELS))+'Label_'+SOURCE_NAME.split('.')[0])

In [40]:
# Data Loading
df = pd.read_csv(os.path.join('.', 'Data', SOURCE_NAME), encoding='utf-8-sig')
df[COLNAME_LABEL] = feature_num_CategBin(df['Rating'], bins=BINS, labels=LABELS)
ds = preprocessing_df2Datasets(MODEL_NAME, df[[COLNAME_LABEL, COLNAME_TARGET]], 
                               label_list=LABELS, val_size=VAL_SIZE, 
                               random_seed=RANDOM_SEED, max_length=WORD_MAXLEN)

,count
Rating,
0,24188
1,6487
-1,30128


Map:   0%|          | 0/24540 [00:00<?, ? examples/s]

Map:   0%|          | 0/6135 [00:00<?, ? examples/s]

Map:   0%|          | 0/30128 [00:00<?, ? examples/s]

In [41]:
# Modeling
trainer, history = modeling_SequenceClassification(MODEL_NAME, 
                                                   ds, sample_size=SAMPLE_SIZE,
                                                   train_epochs=EPOCHS,
                                                   loss=LOSS,
                                                   eval_strategy=EVAL_STRATEGY,
                                                   output_dir=OUTPUT_DIR,
                                                   save_folder=SAVE_PATH_MODEL)    

# Prediction
test_pred = prediction_SequenceClassification(task=TASK, model_path=SAVE_PATH_MODEL, 
                                              ds_test_text=ds['test']['text'])
## Replacing
for idx, pred in zip(df[df[COLNAME_LABEL] == -1].index, test_pred):
    df.loc[idx, COLNAME_LABEL] = pred[0]
    df.loc[idx, COLNAME_LABEL+'_Prob'] = pred[1]
df.to_csv(os.path.join('.', 'Data', SOURCE_NAME.split('.')[0]+'_pred('+MODEL_NAME.split('/')[1]+'_'+str(len(LABELS))+'Label).csv'), 
          encoding='utf-8-sig')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.422700,0.397635,0.852486
2,0.362100,0.447087,0.791361
3,0.285600,0.394001,0.837490
4,0.250300,0.432789,0.840098
5,0.217200,0.545247,0.828036
6,0.179400,0.536966,0.837816
7,0.154700,0.553820,0.836838
8,0.142100,0.563171,0.848900
9,0.127800,0.554462,0.839772
10,0.098600,0.623288,0.841402


Device set to use cuda:0
